In [ ]:
import requests
import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import json

In [9]:

def fetch_and_insert_data(db_user, db_password, db_host, db_port, db_name, table_name):
    """
    Fetch data from the API, preprocess it, and insert it into the specified database table.

    Args:
        db_user (str): The database user name.
        db_password (str): The database password.
        db_host (str): The database host.
        db_port (str): The database port.
        db_name (str): The database name.
        table_name (str): The name of the table to insert data into.

    Returns:
        None
    """
    # Construct the database connection URL
    db_url = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'

    # Create a database engine
    engine = create_engine(db_url)

    # Create the table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS public.{table_name} (
        id varchar(50) NOT NULL,
        date_gmt timestamp NOT NULL,
        modified_gmt timestamp NOT NULL,
        title text NOT NULL,
        slug varchar(255) NULL,
        status varchar(20) NOT NULL,
        "type" varchar(20) NOT NULL,
        link text NOT NULL,
        "content" text NULL,
        excerpt text NULL,
        author jsonb NULL,
        editor text NULL,
        comment_status varchar(20) NOT NULL,
        comments_count int4 DEFAULT 0 NOT NULL,
        "comments" jsonb NULL,
        featured_image jsonb NULL,
        post_images jsonb NULL,
        seo jsonb NULL,
        categories jsonb NULL,
        tags jsonb NULL,
        companies jsonb NULL,
        is_sponsored bool DEFAULT false NOT NULL,
        sponsor jsonb NULL,
        is_partnership bool DEFAULT false NOT NULL,
        external_scripts text NULL,
        show_ads bool DEFAULT true NOT NULL,
        is_subscriber_exclusive bool DEFAULT false NOT NULL,
        is_paywalled bool DEFAULT false NOT NULL,
        is_inappbrowser bool DEFAULT false NOT NULL,
        read_time int4 NOT NULL,
        CONSTRAINT {table_name}_pkey PRIMARY KEY (id),
        CONSTRAINT {table_name}_slug_key UNIQUE (slug)
    );
    """

    with engine.connect() as conn:
        conn.execute(sqlalchemy.text(create_table_query))

    # Fetch data from the API
    url = "https://www.techinasia.com/wp-json/techinasia/2.0/posts"
    headers = {
        'Cache-Control': 'no-cache',
        'User-Agent': 'PostmanRuntime/7.38.0',
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    }

    data = fetch_api_data(url, headers)

    # Create a DataFrame from the data
    df = pd.DataFrame(data)

    # Convert datetime strings to Python datetime objects
    df['date_gmt'] = pd.to_datetime(df['date_gmt'])
    df['modified_gmt'] = pd.to_datetime(df['modified_gmt'])

    # Convert the DataFrame to a list of dictionaries
    records = df.to_dict('records')

    # Insert the data into the table
    insert_data(records, table_name, engine)

def fetch_api_data(url, headers):
    """
    Fetch data from the API and return a list of dictionaries.

    Args:
        url (str): The API URL.
        headers (dict): The headers for the API request.

    Returns:
        list: A list of dictionaries containing the API data.
    """
    data = []

    # Make the initial request
    response = requests.request("GET", url, headers=headers, json={})
    response_data = response.json()

    # Check if the 'posts' key exists in the initial response
    if 'posts' in response_data:
        data.extend(response_data['posts'])

    # Get the total number of pages
    total_pages = response_data.get('total_pages', 1)

    # Loop through the remaining pages
    page = 2
    while page <= total_pages:
        next_url = f"{url}?page={page}"

        response = requests.request("GET", next_url, headers=headers, json={})

        if response.ok:
            response_data = response.json()

            if 'posts' in response_data:
                data.extend(response_data['posts'])
            else:
                print(f"No 'posts' key found in the response for page {page}")
        else:
            error_data = response.json()
            if error_data.get('code') == 'rest_invalid_param' and error_data.get('data', {}).get('params', {}).get('page') == 'Invalid parameter.':
                print("Encountered 'Invalid parameter(s): page' error. Stopping the loop.")
                break
            else:
                print(f"Error fetching page {page}: {error_data}")

        page += 1

    return data

def insert_data(records, table_name, engine):
    """
    Insert data into the specified table using SQLAlchemy.

    Args:
        records (list): A list of dictionaries containing the data to be inserted.
        table_name (str): The name of the table to insert data into.
        engine (sqlalchemy.engine.base.Engine): The SQLAlchemy engine object.

    Returns:
        None
    """
    insert_query = f"""
    INSERT INTO {table_name} (
        id, date_gmt, modified_gmt, title, slug, status, type, link, content, excerpt,
        author, editor, comment_status, comments_count, comments, featured_image,
        post_images, seo, categories, tags, companies, is_sponsored, sponsor,
        is_partnership, external_scripts, show_ads, is_subscriber_exclusive,
        is_paywalled, is_inappbrowser, read_time
    ) VALUES (
        %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s,
        %s, %s, %s, %s, %s, %s, %s
    )
    """

    with engine.connect() as conn:
        for record in records:
            values = (
                record['id'], record['date_gmt'], record['modified_gmt'], record['title'], record['slug'], record['status'],
                record['type'], record['link'], record['content'], record['excerpt'], json.dumps(record['author']), record['editor'],
                record['comment_status'], record['comments_count'], json.dumps(record['comments']), json.dumps(record['featured_image']),
                json.dumps(record['post_images']), json.dumps(record['seo']), json.dumps(record['categories']), json.dumps(record['tags']), json.dumps(record['companies']),
                record['is_sponsored'], json.dumps(record['sponsor']), record['is_partnership'], record['external_scripts'],
                record['show_ads'], record['is_subscriber_exclusive'], record['is_paywalled'], record['is_inappbrowser'],
                record['read_time']
            )
            conn.execute(insert_query, values)
    print("done")

In [10]:
# main.py
# from your_module import fetch_and_insert_data

# if __name__ == "__main__":
db_user = "ghalyrizqimauludin"
db_password = ""
db_host = "localhost"
db_port = "5432"
db_name = "postgres"
table_name = "posts"

fetch_and_insert_data(db_user, db_password, db_host, db_port, db_name, table_name)

Encountered 'Invalid parameter(s): page' error. Stopping the loop.
done
